In [ ]:
import pandas as pd
import numpy as np
import scipy as sp
from datetime import datetime
import MetaTrader5 as mt5
import matplotlib.pyplot as plt
import statsmodels.api as sm

In [ ]:
def get_clean_mt5_data(tickers, interval, n_bars=99999, window = 10, shift= -5):
    datas = []
    for ticker in tickers:
        data = pd.DataFrame(mt5.copy_rates_from(ticker, interval, datetime.now(), n_bars))
        data['time'] = pd.to_datetime(data['time'], unit='s')
        data['symbol'] = np.full(shape=len(data), fill_value=ticker)
        data = data.drop(['real_volume'], axis=1)
        datas.append(data)

    data = pd.concat(datas, axis=0)
    data = data.set_index(['symbol', 'time'])

    # Calculating volatility
    data['returns'] = data['close'].pct_change()
    data['volatility'] = data['returns'].rolling(window=window).std()

    # Calculating extended returns
    data['extended_returns'] = data['returns'].shift(shift)

    # Keep only the volatility and extended_returns columns
    data = data[['volatility', 'extended_returns']]
    data = data.dropna(subset=['volatility','extended_returns'], how='any')

    return data

In [ ]:
tickers = ['GBPUSD','USDJPY']
mt5.initialize()

login_mt5 = 1051534030
mdp_mt5 = 'FG2SF2M74R'
server = 'FTMO-Demo'

mt5.login(login_mt5, mdp_mt5, server)

In [ ]:
data = get_clean_mt5_data(tickers, interval=mt5.TIMEFRAME_H4, n_bars=8000, window=10, shift = -4)

In [ ]:

mediane = np.median(data['volatility'])

mad = np.median(np.abs(data['volatility'] - mediane))

borne_seuil = 3 * mad

valeurs_aberrantes = data['volatility'][np.abs(data['volatility'] - mediane) > borne_seuil]

data = data[~np.isin(data['volatility'], valeurs_aberrantes)]

In [ ]:
volatility_min = np.min(data['volatility'])
volatility_max = np.max(data['volatility'])
extended_returns_min = np.min(data['extended_returns'])
extended_returns_max = np.max(data['extended_returns'])

print("Volatilité - Min :", volatility_min)
print("Volatilité - Max :", volatility_max)
print("Extended Returns - Min :", extended_returns_min)
print("Extended Returns - Max :", extended_returns_max)

Fonction Corrélation

In [ ]:
def calculate_correlations(data):
    correlation_spearman = data['extended_returns'].corr(data['volatility'], method='spearman')
    correlation_kendall = data['extended_returns'].corr(data['volatility'], method='kendall')
    correlation_pearson = data['extended_returns'].corr(data['volatility'], method='pearson')



    data['Constant'] = 1
    y = data['extended_returns']
    X = data[['volatility', 'Constant']]
    model = sm.OLS(y, X)
    results = model.fit()
    coefficients = results.params
    correlation_regression = coefficients[0]

    return correlation_spearman, correlation_kendall, correlation_pearson, correlation_regression




Calcule corrélations (4 méthodes) pour chaque window/shift

In [ ]:



corr_table = pd.DataFrame(columns=["Window/Shift", "Spearman", "Kendall", "Pearson", "Regression"])

for window in range(3, 15):
    for shift in range(-2, -6, -1):
        
        data = get_clean_mt5_data(tickers, interval=mt5.TIMEFRAME_H4, n_bars=8000, window=window, shift=shift)

    
        correlation_spearman, correlation_kendall, correlation_pearson, correlation_regression = calculate_correlations(data)

        
        temp_df = pd.DataFrame({"Window/Shift": f"Window {window} Shift {shift}",
                                "Spearman": correlation_spearman,
                                "Kendall": correlation_kendall,
                                "Pearson": correlation_pearson,
                                "Regression": correlation_regression}, index=[0])

        corr_table = pd.concat([corr_table, temp_df], ignore_index=True)


print(corr_table)


In [ ]:
extended_returns = [10, 20, 30, 40, 50, 60, 70, 80]
volatility = [0.0005, 0.001, 0.0015, 0.0020, 0.0025, 0.003]

plt.scatter(volatility, extended_returns)
plt.xlabel('Volatility')
plt.ylabel('Extended Returns')
plt.title('Relation entre Volatility et Extended Returns')
plt.grid(True)
plt.show()